In [2]:
# Chapter 7.1 예제 3 - 재치있고 비꼬는 AI 어시스턴트
# 퓨샷 프롬프트를 사용해 모델의 응답 스타일을 조정하는 방법

# 필요한 라이브러리 설치
# !pip install langchain==0.2.17 langchain-openai

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

print("OpenAI API 키 설정이 필요합니다.")
print("API 키 발급: https://platform.openai.com/api-keys")

if 'OPENAI_API_KEY' not in os.environ:
    print("\n다음 중 한 가지 방법을 선택하세요:")
    print("1. 아래에서 직접 입력 (안전함 - 화면에 표시되지 않음)")
    print("2. 환경 변수로 설정 (export OPENAI_API_KEY=your-key)")
    
    try:
        api_key = getpass.getpass("\nAPI 키를 입력하세요: ")
        if api_key:
            os.environ['OPENAI_API_KEY'] = api_key
            print("✅ API 키가 설정되었습니다!")
        else:
            print("⚠️ API 키가 입력되지 않았습니다.")
    except KeyboardInterrupt:
        print("\n입력이 취소되었습니다.")
else:
    print("✅ 기존 환경 변수의 API 키를 사용합니다.")

print("\n" + "="*80)

# =============================================================================
# 라이브러리 import 및 기본 설정
# =============================================================================
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

print("🤖 LLM 모델 초기화 완료")

# =============================================================================
# 재치있고 비꼬는 AI 어시스턴트 만들기
# =============================================================================
print("\n" + "="*80)
print("😏 재치있고 비꼬는 AI 어시스턴트 예제")
print("="*80)

print("""
이 예제에서는 퓨샷 프롬프트를 사용해 모델이 비꼬고(sarcastic) 재치 있는 
답변을 하도록 유도하는 방법을 보여줍니다.

⚠️ 주의: 이는 교육 목적의 예제입니다. 실제 서비스에서는 
사용자에게 도움이 되는 친절한 응답을 제공하는 것이 좋습니다.
""")

# 예시 대화 (few-shot examples)
examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    },
    {
        "query": "Why is the sky blue?", 
        "answer": "It's nature's way of preventing eye strain."
    }
]

print("📝 사용할 예시 대화:")
for i, example in enumerate(examples, 1):
    print(f"{i}. 질문: {example['query']}")
    print(f"   답변: {example['answer']}\n")

# 각 예시를 포맷하는 템플릿
example_template = """User: {query}
AI: {answer}"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# 프롬프트의 전후 맥락 설정
prefix = """The following are excerpts from conversations with an AI assistant. The assistant
is typically sarcastic and witty, producing creative and funny responses to users' questions.
Here are some examples:

(다음은 AI 비서와의 대화에서 발췌한 내용이다. 이 비서는 주로 비꼬고 재치 있는, 
창의적이고 재미있는 답변을 사용자에게 제공한다. 다음의 예시를 참고하자.)"""

suffix = """User: {query}
AI:"""

# FewShotPromptTemplate 생성
sarcastic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

print("🔍 생성된 프롬프트 확인:")
sample_prompt = sarcastic_prompt.format(query="How can I learn quantum computing?")
print(sample_prompt)
print("\n" + "-"*60)

# =============================================================================
# 최신 방식으로 실행
# =============================================================================
print("\n방법: RunnableSequence 사용 (최신 LangChain)")

# 최신 방식: prompt | llm 파이프라인
chain = sarcastic_prompt | llm

# 테스트 질문들
test_queries = [
    "How can I learn quantum computing?",
    "What's the meaning of life?",
    "How do I fix my computer?",
    "Why do I keep procrastinating?",
    "What's the best way to learn AI?"
]

try:
    print("\n🎭 재치있는 AI의 답변들:")
    print("="*50)
    
    for query in test_queries:
        response = chain.invoke({"query": query})
        print(f"👤 사용자: {query}")
        print(f"🤖 AI: {response.content}")
        print("-" * 40)
        
except Exception as e:
    print(f"⚠️ 실행 중 오류: {e}")
    print("💡 API 키가 올바르게 설정되었는지 확인하세요.")

# =============================================================================
# 기존 방식과 비교 (참고용)
# =============================================================================
print("\n" + "="*80)
print("📚 기존 방식 vs 새로운 방식 비교")
print("="*80)

print("""
# 원서의 기존 방식 (Deprecated):
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=sarcastic_prompt)  # ⚠️ Deprecated
response = chain.run({"query": "your question"})    # ⚠️ Deprecated

# 현재 권장 방식:
chain = sarcastic_prompt | llm                      # ✅ 권장
response = chain.invoke({"query": "your question"}) # ✅ 권장

주요 변경사항:
1. LLMChain 클래스 → RunnableSequence (| 연산자)
2. chain.run() 메소드 → chain.invoke() 메소드
3. 더 간결하고 직관적인 문법
""")

# =============================================================================
# 스타일 변경 실험
# =============================================================================
print("\n" + "="*80)
print("🎨 다른 스타일로 실험해보기")
print("="*80)

# 친절한 버전
friendly_examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Practice regularly, read good code, and don't be afraid to ask questions. You've got this!"
    },
    {
        "query": "Why is the sky blue?",
        "answer": "It's due to Rayleigh scattering of sunlight in the atmosphere. Blue light scatters more than other colors."
    }
]

friendly_prefix = """The following are excerpts from conversations with a helpful AI assistant. The assistant
is kind, encouraging, and provides accurate information to help users learn and grow.
Here are some examples:"""

friendly_prompt = FewShotPromptTemplate(
    examples=friendly_examples,
    example_prompt=example_prompt,
    prefix=friendly_prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

friendly_chain = friendly_prompt | llm

print("비교 테스트 - 같은 질문, 다른 스타일:")

test_question = "How can I learn quantum computing?"

try:
    # 비꼬는 버전
    sarcastic_response = chain.invoke({"query": test_question})
    
    # 친절한 버전  
    friendly_response = friendly_chain.invoke({"query": test_question})
    
    print(f"\n질문: {test_question}")
    print(f"\n😏 비꼬는 AI: {sarcastic_response.content}")
    print(f"\n😊 친절한 AI: {friendly_response.content}")
    
except Exception as e:
    print(f"⚠️ 비교 테스트 중 오류: {e}")

# =============================================================================
# 실제 활용 시 고려사항
# =============================================================================
print("\n" + "="*80)
print("💡 실제 활용 시 고려사항")
print("="*80)

print("""
1. 사용자 경험 우선:
   - 재미있지만 도움이 되지 않는 답변보다는 유용한 정보 제공이 중요
   - 사용자의 감정 상태나 상황을 고려

2. 브랜드 톤앤매너:
   - 비즈니스 맥락에 맞는 적절한 톤 설정
   - 일관된 브랜드 개성 유지

3. 안전성:
   - 부적절하거나 해로운 내용 생성 방지
   - 사용자를 모독하거나 상처주는 답변 피하기

4. 맥락 인식:
   - 진지한 질문에는 진지하게 답변
   - 상황에 맞는 적절한 응답 수준 조절

5. 퓨샷 예시 품질:
   - 원하는 스타일을 잘 보여주는 고품질 예시 사용
   - 다양한 상황을 커버하는 예시 준비
""")

print("\n🎉 Chapter 7.1 예제 3 완료!")
print("📚 다음 단계: 7.2절 퓨샷 프롬프트와 예시 선택기의 고급 기능")

OpenAI API 키 설정이 필요합니다.
API 키 발급: https://platform.openai.com/api-keys
✅ 기존 환경 변수의 API 키를 사용합니다.

🤖 LLM 모델 초기화 완료

😏 재치있고 비꼬는 AI 어시스턴트 예제

이 예제에서는 퓨샷 프롬프트를 사용해 모델이 비꼬고(sarcastic) 재치 있는 
답변을 하도록 유도하는 방법을 보여줍니다.

⚠️ 주의: 이는 교육 목적의 예제입니다. 실제 서비스에서는 
사용자에게 도움이 되는 친절한 응답을 제공하는 것이 좋습니다.

📝 사용할 예시 대화:
1. 질문: How do I become a better programmer?
   답변: Try talking to a rubber duck; it works wonders.

2. 질문: Why is the sky blue?
   답변: It's nature's way of preventing eye strain.

🔍 생성된 프롬프트 확인:
The following are excerpts from conversations with an AI assistant. The assistant
is typically sarcastic and witty, producing creative and funny responses to users' questions.
Here are some examples:

(다음은 AI 비서와의 대화에서 발췌한 내용이다. 이 비서는 주로 비꼬고 재치 있는, 
창의적이고 재미있는 답변을 사용자에게 제공한다. 다음의 예시를 참고하자.)

User: How do I become a better programmer?
AI: Try talking to a rubber duck; it works wonders.

User: Why is the sky blue?
AI: It's nature's way of preventing eye strain.

User: How can I learn quantum computing?
AI:
